In [1]:
import os
import tempfile
import shutil
import urllib
import zipfile
import pandas as pd
import requests
import sys
import json

In [2]:
behaviors = pd.read_table(
    '~/MIND/behaviors.tsv',
    header=None,
    names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

news = pd.read_table('~/MIND/news.tsv',
              header=None,
              names=[
                  'id', 'category', 'subcategory', 'title', 'abstract', 'url',
                  'title_entities', 'abstract_entities'
              ])

In [3]:
#CONVERT DATAS
behaviors['time'] = pd.to_datetime(behaviors['time'], format='%m/%d/%Y %H:%M:%S %p')
# behaviors['impressions'] = pd.to_datetime(behaviors['time'], format='%m/%d/%Y %H:%M:%S %p')
behaviors['history'] = behaviors['history'].str.split()
behaviors['impressions'] = behaviors['impressions'].str.split()

In [37]:
model = None
validation = behaviors.loc[behaviors['user_id'] == 'U82600']

cont=1

querys = []

for index, row in validation.iterrows():
    articles = []
    query = {
        'query' : 'Q{}'.format(cont),
        'used': True,
        'state' : 'Done',
        "userId": "nogueira.matheus2109@gmail.com",
        
    }
    for item in row['impressions']:
        _id,label  = item.split('-')
        if label == '0':
            label = False
        else:
            label = True
        n = news[news['id'] == _id]
        article = {
            'abstract':str(n.abstract.values[0]),
            'title':str(n.title.values[0]),
            'url':str(n.url.values[0])
        }
        r = requests.post("http://192.168.10.34:8000/api/insert/article/",json=article)
        
        a_id = r.json()
        articles.append([a_id,label,None])
        
    query['articles'] = articles
    querys.append(query)
    cont+=1
    print('---------------')
    
job = {
    'querys':[q['query'] for q in querys],
    'user' : 'nogueira.matheus2109@gmail.com',
    'used': False,
    
}
print(job)
r = requests.post("http://192.168.10.34:8000/api/insert/job/",json=job)
job_id = r.json()['_id']
for query in querys:
    query['jobId'] = job_id
    r = requests.post("http://192.168.10.34:8000/api/insert/query/",json=job)

---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
{'querys': ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12'], 'user': 'nogueira.matheus2109@gmail.com', 'used': False}


In [103]:
behaviors.set_index(["user_id","history"]).count(level="user_id")['time']

user_id
U1        1
U10       1
U10000    3
U10002    3
U10004    2
         ..
U9990     1
U9994     1
U9996     4
U9998     1
U9999     2
Name: time, Length: 50000, dtype: int64

In [31]:
behaviors.sort_values(by=['time'])

,impression_id,user_id,time,history,impressions
67130,67131,U28754,2019-11-15 01:00:00,"[N55189, N60206, N2203, N51706, N20121, N9155,...","[N6400-0, N29091-0, N24802-1, N53572-0, N29862-0]"
14543,14544,U48633,2019-11-15 01:00:00,"[N36133, N1150, N55189, N38118, N56499, N43677...","[N33176-0, N59933-0, N31884-0, N15347-0, N7618..."
19859,19860,U53371,2019-11-15 01:00:01,"[N44399, N14302, N6264, N39235, N23526, N60000...","[N24802-0, N5472-0, N6400-1, N29091-0, N29862-..."
32646,32647,U88673,2019-11-15 01:00:01,NaN,"[N5472-0, N55405-0, N55237-0, N42844-0, N6916-..."
36526,36527,U72398,2019-11-15 01:00:01,"[N47954, N45794, N59704, N35452, N55922, N4064...","[N5507-0, N30283-0, N44698-0, N63913-0, N53346..."
...,...,...,...,...,...
39312,39313,U62905,2019-11-15 12:59:57,"[N60615, N55805, N53234, N6233, N64408, N22293...","[N6400-0, N29862-1, N53572-0, N11390-0, N29091..."
27051,27052,U65939,2019-11-15 12:59:58,"[N1150, N10059, N20121, N19630, N58641, N25577...","[N24802-0, N5940-0, N31958-0, N53572-0, N11930..."
71723,71724,U58613,2019-11-15 12:59:59,"[N34004, N36312, N12469, N29026, N62058, N49990]","[N5472-0, N29091-0, N48740-0, N11390-0, N6400-..."
32149,32150,U28891,2019-11-15 12:59:59,"[N37075, N32284, N30507, N7898, N41797, N23043...","[N51470-0, N11390-0, N24802-1, N29862-0, N5523..."


In [84]:
behaviors.dtypes

impression_id             int64
user_id                  object
time             datetime64[ns]
history                  object
impressions              object
dtype: object